In [ ]:
!pip install pycountry

In [ ]:
!pip install plotly

In [10]:
import os
import pandas as pd
import matplotlib
import numpy as np
import plotly
import logging
logging.getLogger().setLevel(logging.DEBUG)

### Carregar os dados

Faça a Leitura dos Arquivos das <a href="https://www.kaggle.com/arjunprasadsarkhel/2021-olympics-in-tokyo">Olimpíadas de Tokyo 2020</a> usando a pandas

In [159]:
paths = ["data/"+path for path in os.listdir('data')]
df_medals = pd.read_excel(paths[0])
df_athletes = pd.read_excel(paths[1])
df_teams = pd.read_excel(paths[2])
df_coaches = pd.read_excel(paths[3])
df_entries = pd.read_excel(paths[4])

### Fazer algumas alterações

Crie uma coluna 'Pontuação' no DataFrame referente a Medalhas (Atribua 10 pontos para cada medalha de ouro, 5 para cada de prata e 1 para cada de bronze)

In [160]:
df_medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [161]:
# Renomear as colunas
df_medals.columns = ['Rank', 'País', 'Ouro', 'Prata', 'Bronze', 'Total', 'Rank do Total']

In [181]:
# Criar coluna de pontuações
df_medals["Pontuação"] = df_medals['Ouro']*10 + df_medals['Prata']*5 + df_medals['Bronze']

### Gerar um gráfico de linhas comparando medalhas de ouro com pontuação

Gere um gráfico de Pontuação dos Países usando o plotly

In [183]:
import plotly.express as px
fig = px.line(x='Ouro', y='Pontuação', data_frame=df_medals, title='Medalhas vs pontuações')
fig.show()

### Graficar um mapa de medalhas de ouro por país

In [52]:
import pycountry

In [163]:
# Renomear a federação russa
df_medals['País_corregido'] = df_medals['País'].apply(lambda x: "Russian Federation" if x == 'ROC'
                                            else "United Kingdom" if x == 'Great Britain'
                                            else "Korea, Republic of" if x == 'Republic of Korea'
                                            else "China" if x == "Chinese Taipei" or x == "Hong Kong, China"
                                            else "Serbia" if x == "Kosovo"
                                            else "Bolivarian Republic of Venezuela" if x == "Venezuela"
                                            else x)
# Criar lista de países
input_countries = df_medals['País_corregido'].unique()

In [175]:
# Aggregar coluna de códigos de país
codes = {}
for country in list(pycountry.countries):
    try:
        codes[country.official_name] = country.alpha_3
        codes[country.name] = country.alpha_3
    except:
        codes[country.name] = country.alpha_3
codes_dct = {country:codes[country] for country in input_countries if country in codes.keys()}
df_medals['Código'] = df_medals['País_corregido'].map(codes_dct)

In [184]:
# Graficar
import plotly.graph_objects as go
fig = go.Figure(data=go.Choropleth(
    locations = df_medals['Código'],
    z = df_medals['Pontuação'],
    text = df_medals['País_corregido'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Total de pontuações por país',
))
fig.show()

### Salvar o DataFrame para um arquivo de Excel 2007-365

Salve o novo DataFrame no formato xlsx.

In [186]:
df_medals.to_excel("data/Medals (modified).xlsx", index = False)